# Step 2: Reaction Rules Extraction

This tutorial demonstrates how to extract reaction rules from reaction data in ``SynPlanner``

## Basic recommendations

1. The specificity of extracted reaction rules can be adjusted by the configuration of the extraction protocol

3. The extracted reaction rules are stored as the pickled list of ``CGRtools`` ``ReactionContainer`` objects. Currently, there is no implementation of the conversion of the extracted reaction rules to portable representation (e.g. reaction SMARTS). 

## 1. Set up input and output data locations

The ``SynPlanner`` input data will be downloaded from the ``HuggingFace`` repository to the specified directory.

In [1]:
import os
import pickle
import shutil
from pathlib import Path
from synplan.utils.loading import download_all_data

# download SynPlanner data
data_folder = Path("synplan_data").resolve()
download_all_data(save_to=data_folder)

# results folder
results_folder = Path("tutorial_results").resolve()
results_folder.mkdir(exist_ok=True)

# input data
# use default filtered data from tutorial folder or replace with custom data prepared with data curation tutorial
filtered_data_path = results_folder.joinpath("uspto_filtered.smi") 

# output_data
reaction_rules_path = results_folder.joinpath("uspto_reaction_rules.pickle")

Fetching 25 files:   0%|          | 0/25 [00:00<?, ?it/s]

## 2. Reaction rules extraction

The reaction rule extraction protocol in ``SynPlanner`` includes several steps (e.g. reaction center identification and specification, reaction rule validation, etc.)

More details about reaction rule extraction protocol in ``SynPlanner`` can be found in <a href="https://synplanner.readthedocs.io/en/latest/methods/extraction.html">official documentation</a>.

### Extraction configuration

The next step is to configure the reaction rule extraction process. We do this using the `RuleExtractionConfig` class in ``SynPlanner``. This class allows for the specification of various parameters and settings for the reaction rule extraction process.

More details about reaction rule extraction configuration in ``SynPlanner`` can be found in <a href="https://synplanner.readthedocs.io/en/latest/configuration/extraction.html">official documentation</a>.

In [2]:
from synplan.utils.config import RuleExtractionConfig
from synplan.chem.reaction_rules.extraction import extract_rules_from_reactions

extraction_config = RuleExtractionConfig(
    min_popularity=3,
    environment_atom_count=1,
    multicenter_rules=True,
    include_rings=False,
    keep_leaving_groups=True,
    keep_incoming_groups=False,
    keep_reagents=False,
    atom_info_retention={
        "reaction_center": {
            "neighbors": True,  # retains information about neighboring atoms to the reaction center
            "hybridization": True,  # preserves the hybridization state of atoms at the reaction center
            "implicit_hydrogens": False,  # includes data on implicit hydrogen atoms attached to the reaction center
            "ring_sizes": False,  # keeps information about the sizes of rings that reaction center atoms are part of
        },
        "environment": {
            "neighbors": False,  # retains information about neighboring atoms to the atoms in the environment of the reaction center
            "hybridization": False,  # preserves the hybridization state of atoms in the environment
            "implicit_hydrogens": False,  # includes data on implicit hydrogen atoms attached to atoms in the environment
            "ring_sizes": False,  # keeps information about the sizes of rings that environment atoms are part of
        },
    },
)

### Running extraction

After configuring the rule extraction settings in ``SynPlanner``, the next step is to apply these configurations to extract reaction rules from the reaction data. This is achieved using the `extract_rules_from_reactions` function.

In [3]:
extract_rules_from_reactions(
    config=extraction_config,  # the configuration settings for rule extraction
    reaction_data_path=filtered_data_path,  # path to the reaction data file
    reaction_rules_path=reaction_rules_path,  # path to the pickle file where the extracted reaction rules will be stored
    num_cpus=4,
    batch_size=100,
)

Number of reactions processed: 1019304 [1:03:45]


Number of extracted reaction rules: 24422


The extracted reaction rules can be loaded and visually inspected (the reaction rules are sorted by popularity):

In [4]:
with open(reaction_rules_path, "rb") as f:
    reaction_rules_list = [i[0] for i in pickle.load(f)]
for rule in reaction_rules_list:
    rule.clean2d()

In [5]:
reaction_rules_list[0]

In [6]:
reaction_rules_list[1]

In [7]:
reaction_rules_list[100]

## Results

If the tutorial is executed successfully, you will get in the results folder three reaction data files (from reaction curation tutorial) and corresponding extracted reaction rules:
- original reaction data
- standardized reaction data
- filtered reaction data
- extracted reaction rules

In [8]:
sorted(Path(results_folder).iterdir(), key=os.path.getmtime, reverse=False)

[PosixPath('/home1/dima/synplanner/tutorials/tutorial_results/uspto_original.smi'),
 PosixPath('/home1/dima/synplanner/tutorials/tutorial_results/uspto_standardized.smi'),
 PosixPath('/home1/dima/synplanner/tutorials/tutorial_results/uspto_filtered.smi'),
 PosixPath('/home1/dima/synplanner/tutorials/tutorial_results/uspto_reaction_rules.pickle')]